## Lines from `Grid` objects

Grids that have been post-processed through a photoionisation code (e.g. `Cloudy`) contain information on emission lines.
These can be loaded like regular grids, but there are a number of additional methods for working with lines, as demonstrated in these examples.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import synthesizer.line_ratios as line_ratios
from synthesizer.grid import Grid
from synthesizer.line import (
    get_diagram_labels,
)

Let's first introduce the `line_ratios` module. This contains a set of useful definitions.

In [ ]:
# the ID of H-alpha
print(line_ratios.Ha)

# the available in-built line ratios ...
print(line_ratios.available_ratios)

# ... and diagrams.
print(line_ratios.available_diagrams)

Next let's initialise a grid:

In [ ]:
grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

We can easily get a list of the available lines:

In [ ]:
print(grid.available_lines)

This is also reported if we give use the `print` function on a grid directly:

In [ ]:
print(grid)

To demonstrate, we choose some age and metallicity and extract the spectra at that grid point. We can then get information on a single line, in this case H-$\beta$.

In [ ]:
log10age = 6.0  # log10(age/yr)
metallicity = 0.01

# find nearest grid point
grid_point = grid.get_grid_point((log10age, metallicity))

line_id = line_ratios.Hb
line = grid.get_line(grid_point, line_id)
print(line)

We can do this for a combination of lines (e.g. a doublet).

Note that this sums the contribution of each line; if you want separate lines, use the `get_lines` method described below.

In [ ]:
line = grid.get_line(
    grid_point, [line_ratios.Hb, line_ratios.O3r, line_ratios.O3b]
)
print(line)

We can also create a `LineCollection`, a collection of lines which have methods for calculating ratios and diagrams. By default this will create a collection for all available lines, but you can also specify which lines you want.

In [ ]:
lines = grid.get_lines(grid_point)
print(lines)

We can measure some predefined line ratios, or loop over all pre-defined ratios:

In [ ]:
ratio_id = "BalmerDecrement"
ratio = lines.get_ratio(ratio_id)
print(f"{ratio_id}: {ratio:.2f}")

for ratio_id in lines.available_ratios:
    ratio = lines.get_ratio(ratio_id)
    print(f"{ratio_id}: {ratio:.2f}")

We can also easily measure the ratio of an arbitrary set of lines:

In [ ]:
lines.get_ratio(["Ne 4 1601.45A", "He 2 1640.41A"])
lines.get_ratio(["Ne 4 1601.45A, He 2 1640.41A", "O 3 1660.81A"])

To show the dependence on stellar metallicity we can loop over the metallicity grid:

In [ ]:
ratio_id = "R23"
ia = 0  # 1 Myr old for test grid
ratios = []
for iZ, Z in enumerate(grid.metallicity):
    grid_point = (ia, iZ)
    lines = grid.get_lines(grid_point)
    ratios.append(lines.get_ratio(ratio_id))

Zsun = grid.metallicity / 0.0124
plt.plot(Zsun, ratios)
plt.xlim([0.01, 1])
plt.ylim([1, 20])
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$Z/Z_{\odot}$")
plt.ylabel(rf"{ratio_id}")
# plt.ylabel(rf'${get_ratio_label(ratio_id)}$')
plt.show()

We can also generate diagrams using pairs of line ratios, such as the famous Baldwin, Phillips & Terlevich (BPT) diagram.

``line_ratios`` also contains some classification regions (e.g. [Kewley+13](https://ui.adsabs.harvard.edu/abs/2013ApJ...774L..10K/abstract) and [Kauffmann+03](https://ui.adsabs.harvard.edu/abs/2003MNRAS.346.1055K/abstract)) that we can plot:

In [ ]:
diagram_id = "BPT-NII"
ia = 0  # 1 Myr old for test grid
x = []
y = []
for iZ, Z in enumerate(grid.metallicity):
    grid_point = (ia, iZ)
    lines = grid.get_lines(grid_point)
    x_, y_ = lines.get_diagram(diagram_id)
    x.append(x_)
    y.append(y_)


# plot the Kewley SF/AGN dividing line

logNII_Ha = np.arange(-2.0, 1.0, 0.01)
logOIII_Hb = line_ratios.get_bpt_kewley01(logNII_Ha)
plt.plot(10**logNII_Ha, 10**logOIII_Hb, c="k", lw="2", alpha=0.3)

plt.plot(x, y)
plt.xlim([0.01, 10])
plt.ylim([0.05, 20])
plt.xscale("log")
plt.yscale("log")

# grab x and y labels, this time use "fancy" label ids
xlabel, ylabel = get_diagram_labels(diagram_id)

plt.xlabel(rf"${xlabel}$")
plt.ylabel(rf"${ylabel}$")
plt.show()